### load  package

In [1]:
import os
root_dir ='../'
os.chdir(root_dir)

In [5]:
model_dir = './model_checkpoint'
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
else:
    print(f"{model_dir} is already existed !")

./model_checkpoint is already existed !


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
from IPython.display import display

import copy
import pickle
pd.set_option('display.max_columns', 50)

### load data

In [5]:
train_path ='./data/trainset-281-29.xlsx'
train_df = pd.read_excel(train_path)

predict_path ='./data/testset-for-participants.xlsx'
predict_df = pd.read_excel(predict_path)

### missing data exploratory

In [6]:
for df in [train_df,predict_df]:
    for col in df.columns:
        df[col] =df[col].apply(lambda x : None if str.lower(str(x)) in ['none','non','nan'] else x)

In [7]:
columns =['feature_name','missing_num','type']

train_agg =train_df.isnull().sum(axis=0).reset_index().sort_values(by=0)
train_agg.rename(columns={0:"train_missing"},inplace=True)

predict_agg = predict_df.isnull().sum(axis=0).reset_index().sort_values(by=0)
predict_agg.rename(columns={0:"predict_missing"},inplace=True)

miss_agg = pd.merge(train_agg,predict_agg,on='index')

In [8]:
miss_agg

,index,train_missing,predict_missing
0,TRAN_AMT,0,0
1,TRAN_TYPE_CD,0,0
2,ACTN_INTNL_TXT,0,0
3,ACTN_CD,0,0
4,TRAN_DT,0,0
5,TRAN_TS,0,0
6,CUST_SINCE_DT,0,0
7,CUST_ZIP,0,0
8,ACTVY_DT,0,0
9,AUTHC_PRIM_TYPE_CD,0,0


##### relationship between missing value and fraud

In [9]:
col_missing =['CUST_STATE','AUTHC_SCNDRY_STAT_TXT',
              'DVC_TYPE_TXT','STATE_PRVNC_TXT','RGN_NAME',
             'CARR_NAME','PWD_UPDT_TS','PH_NUM_UPDT_TS']

label ='FRAUD_NONFRAUD'

In [10]:
missing_df = copy.deepcopy(train_df[col_missing+[label]])
missing_df['num'] =1

aggs =pd.DataFrame()
for col in col_missing:
    missing_df[col] = missing_df[col].apply(lambda x: 'missing' if x is None else 'filled')
    agg = pd.pivot_table(data=missing_df,index=col,columns=label,values='num',aggfunc='sum',fill_value=0)
    agg.reset_index(inplace=True)
    agg.rename(columns={col:"value"},inplace=True)
    agg['feature_name'] =col
    aggs = pd.concat([aggs,agg])

In [11]:
seq_cols =['feature_name','value', 'Fraud', 'Non-Fraud']
aggs[seq_cols]

FRAUD_NONFRAUD,feature_name,value,Fraud,Non-Fraud
0,CUST_STATE,filled,4164,9800
1,CUST_STATE,missing,0,36
0,AUTHC_SCNDRY_STAT_TXT,filled,4164,9762
1,AUTHC_SCNDRY_STAT_TXT,missing,0,74
0,DVC_TYPE_TXT,filled,3215,9024
1,DVC_TYPE_TXT,missing,949,812
0,STATE_PRVNC_TXT,filled,2193,9091
1,STATE_PRVNC_TXT,missing,1971,745
0,RGN_NAME,filled,2195,9096
1,RGN_NAME,missing,1969,740


##### summery

from the table above, we could get the following two conclusions:

1. missing value increase the  fraudulent incident：['DVC_TYPE_TXT'，'STATE_PRVNC_TXT','RGN_NAME','CARR_NAME']


2. missig value decrease the fraudulent incident:['PWD_UPDT_TS','PH_NUM_UPDT_TS']

##### missing value  treatment

In [12]:
missing_value =-999

missing_label ='missing'

In [13]:
train_df.fillna(-999,inplace=True)

In [14]:
predict_df.fillna(-999,inplace=True)

### feature  exploratory

In [15]:
from data_processing import processing

p_conn =processing.processing()

####  duplicated features

In [16]:
duplicate_cols =['ACTVY_DT','TRAN_DT','CUST_ZIP']

####  single- dimension features

In [17]:
single_dim_cols =['ACTN_CD','ACTN_INTNL_TXT','TRAN_TYPE_CD']

####  discreted features

In [18]:
discreted_cols =['CARR_NAME','RGN_NAME','STATE_PRVNC_TXT','CUST_STATE',
                 'ALERT_TRGR_CD','DVC_TYPE_TXT','AUTHC_PRIM_TYPE_CD','AUTHC_SCNDRY_STAT_TXT']

In [19]:
train_value_unique =train_df[discreted_cols].nunique().reset_index()
train_value_unique.rename(columns={0:"training-set-value_num"},inplace=True)

predict_value_unique =predict_df[discreted_cols].nunique().reset_index()

predict_value_unique.rename(columns={0:"perdicting-set-value_num"},inplace=True)

value_unique = pd.merge(train_value_unique,predict_value_unique,on='index')

In [20]:
value_unique

,index,training-set-value_num,perdicting-set-value_num
0,CARR_NAME,555,387
1,RGN_NAME,20,19
2,STATE_PRVNC_TXT,127,104
3,CUST_STATE,49,49
4,ALERT_TRGR_CD,2,2
5,DVC_TYPE_TXT,5,5
6,AUTHC_PRIM_TYPE_CD,5,5
7,AUTHC_SCNDRY_STAT_TXT,4,4


##### summery

The value number of each column in the predicting dataset is concluded in the training datset

##### value more than 10

In [21]:
discreted_cols_over_10 =['CARR_NAME','RGN_NAME','STATE_PRVNC_TXT','CUST_STATE']

In [22]:
# reduce dimension with risk ratio
risk_res ={}

for idx,col in enumerate(discreted_cols_over_10):
    print(f"...{col}")
    target_col = f"{col}_bin_10_feature"
    train_df,value_map =p_conn.risk_cls(train_df,col)
    risk_res[col] =value_map
    df_count = p_conn.agg_fn(train_df,target_col)
    df_count.columns =['value'] + [f"{col}|{i}" for i in df_count.columns if i !=target_col]
    if idx ==0:
        df_counts =df_count
    else:
        df_counts =pd.merge(df_counts,df_count,on='value',how='outer')
        predict_df[f"{col}_bin_10_feature"] = predict_df[col].apply(lambda x : value_map.get(x))
    
df_counts.fillna(0,inplace=True)

pickle.dump(risk_res, open(f"{model_dir}/risk_map.pkl", "wb"))

...CARR_NAME
...RGN_NAME
...STATE_PRVNC_TXT
...CUST_STATE


In [23]:
df_counts

,value,CARR_NAME|Fraud,CARR_NAME|Non-Fraud,RGN_NAME|Fraud,RGN_NAME|Non-Fraud,STATE_PRVNC_TXT|Fraud,STATE_PRVNC_TXT|Non-Fraud,CUST_STATE|Fraud,CUST_STATE|Non-Fraud
0,high-risk,925.0,250.0,476.0,330.0,773.0,477.0,527.0,383.0
1,missing,1969.0,740.0,1969.0,740.0,1971.0,745.0,0.0,36.0
2,no-risk,0.0,550.0,0.0,7.0,0.0,123.0,0.0,9.0
3,risk<0.1,190.0,2781.0,0.0,0.0,0.0,0.0,1.0,12.0
4,risk<0.2,671.0,4490.0,674.0,6458.0,674.0,6458.0,0.0,0.0
5,risk<0.3,10.0,39.0,286.0,1060.0,306.0,1297.0,304.0,1095.0
6,risk<0.4,258.0,769.0,98.0,321.0,0.0,0.0,2195.0,6504.0
7,risk<0.5,39.0,89.0,0.0,0.0,136.0,319.0,92.0,203.0
8,risk<0.6,3.0,5.0,0.0,0.0,25.0,44.0,413.0,763.0
9,risk<0.7,41.0,62.0,225.0,347.0,95.0,150.0,245.0,390.0


##### value less than 10

In [24]:
discreted_cols_below_10 =['ALERT_TRGR_CD','DVC_TYPE_TXT','AUTHC_PRIM_TYPE_CD','AUTHC_SCNDRY_STAT_TXT']

In [25]:
df_counts =pd.DataFrame()
for col in discreted_cols_below_10:
    print(f"...{col}")
    df_count = p_conn.agg_fn(train_df,col)
    df_count.sort_values(by='Fraud',ascending=False,inplace=True)
    df_count['feature'] = col
    df_count.rename(columns={col:"value"},inplace=True)
    df_counts =pd.concat([df_counts,df_count])
    
seq_cols = ['feature','value','Fraud','Non-Fraud']
display(df_counts[seq_cols])

...ALERT_TRGR_CD
...DVC_TYPE_TXT
...AUTHC_PRIM_TYPE_CD
...AUTHC_SCNDRY_STAT_TXT


FRAUD_NONFRAUD,feature,value,Fraud,Non-Fraud
1,ALERT_TRGR_CD,ONLN,2568,4322
0,ALERT_TRGR_CD,MOBL,1596,5514
2,DVC_TYPE_TXT,MOBILE,1581,7215
1,DVC_TYPE_TXT,DESKTOP,1489,1497
0,DVC_TYPE_TXT,-999,949,812
4,DVC_TYPE_TXT,TABLET,77,161
3,DVC_TYPE_TXT,PHONE,68,151
4,AUTHC_PRIM_TYPE_CD,UN_PWD,3371,7165
2,AUTHC_PRIM_TYPE_CD,FACE_ID,521,1432
3,AUTHC_PRIM_TYPE_CD,TOUCH_ID,155,546


##### summery:

1. From the table above, we infer ALERT_TRGR_CD and AUTHC_PRIM_TYPE_CD have little to do with the classification. Later, we will use random forest to identify the  conjecture.

#### datetime formate features

In [26]:
ts_cols =['PWD_UPDT_TS','PH_NUM_UPDT_TS', 'TRAN_TS']

#####  operating hour 

In [27]:
for idx,col in enumerate(ts_cols):
    target_col = f"{col}_hour"
    train_df = p_conn.operate_hour(train_df,col)
    df_count =p_conn.agg_fn(train_df,target_col)
    df_count.rename(columns={target_col:"value"},inplace=True)
    df_count.columns =[i if i =='value' else f"{col}_{i}" for i in df_count.columns ]
    
    if idx ==0:
        df_counts =df_count
    else:
        df_counts =pd.merge(df_counts,df_count,on='value',how='outer')

display(df_counts)

,value,PWD_UPDT_TS_Fraud,PWD_UPDT_TS_Non-Fraud,PH_NUM_UPDT_TS_Fraud,PH_NUM_UPDT_TS_Non-Fraud,TRAN_TS_Fraud,TRAN_TS_Non-Fraud
0,missing,536,2589,1348,5713,0,0
1,bad-case,7,0,502,0,0,0
2,0-4,189,2186,122,234,266,526
3,5-8,516,1320,360,1921,508,1593
4,8-11,615,1160,557,544,1091,2188
5,11-13,1104,601,367,384,921,1387
6,13-17,725,1101,408,590,1013,2119
7,17-19,286,415,82,199,207,1021
8,19-22,160,401,69,220,131,883
9,22-24,26,63,349,31,27,119


##### summery:

The table above shows that:

1. there are several wrong items, like 1/0/2021, in PWD_UPDT_TS and PH_NUM_UPDT_TS_hour, will cause fraudulent  definitely.


2. there seems no relationship between transaction time and fraud, while update phone number and pawwword range from morning to midnight have different fraudulent risk. For example , updating password bewteeen 11AM to 13PM has high fraudulent risk.

##### gap between updating operation, like passwaor or phone number,and transaction 

In [28]:
train_df['PWD_UPDT_TS_day'] = train_df.apply(lambda x : p_conn.timedelta_day(x.TRAN_TS,x.PWD_UPDT_TS),axis =1)    
train_df['PH_NUM_UPDT_TS_day'] = train_df.apply(lambda x : p_conn.timedelta_day(x.TRAN_TS,x.PH_NUM_UPDT_TS),axis =1)    


df_counts =pd.DataFrame()
for idx,col in enumerate(['PWD_UPDT_TS','PH_NUM_UPDT_TS']):
    target_col =f"{col}_day"
    df_count =p_conn.agg_fn(train_df,target_col)
    df_count.rename(columns={target_col:"value"},inplace=True)
    df_count.columns =[i if i =='value' else f"{col}_{i}" for i in df_count.columns ]
    
    if idx ==0:
        df_counts =df_count
    else:
        df_counts =pd.merge(df_counts,df_count,on='value',how='outer')

display(df_counts)

,value,PWD_UPDT_TS_Fraud,PWD_UPDT_TS_Non-Fraud,PH_NUM_UPDT_TS_Fraud,PH_NUM_UPDT_TS_Non-Fraud
0,bad-case,7,0,502,0
1,in_30_days,206,141,37,36
2,in_half_year,591,752,202,241
3,in_one_year,690,1157,711,594
4,missing,536,2589,1348,5713
5,operation_after_transaction,1105,818,248,211
6,over_one_year,1029,4379,1116,3041


#### continuous features

### feature importance

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [30]:
continuous_features =['TRAN_AMT', 'ACCT_PRE_TRAN_AVAIL_BAL', 'CUST_AGE', 'OPEN_ACCT_CT','WF_dvc_age']

origin_discrete_cols =['ALERT_TRGR_CD','DVC_TYPE_TXT','AUTHC_PRIM_TYPE_CD','AUTHC_SCNDRY_STAT_TXT']

manul_discrete_cols =['CARR_NAME_bin_10_feature', 'RGN_NAME_bin_10_feature',
                      'STATE_PRVNC_TXT_bin_10_feature', 'CUST_STATE_bin_10_feature',
                      'PWD_UPDT_TS_hour', 'PH_NUM_UPDT_TS_hour', 'TRAN_TS_hour',
                      'PWD_UPDT_TS_day', 'PH_NUM_UPDT_TS_day']

In [31]:
values_set = set()

replace_map ={"MOBL":'MOBILE','PHONE':"MOBILE",missing_value:missing_label}

for col in origin_discrete_cols+ manul_discrete_cols:
    values = train_df[col].unique().tolist()
    values_set.update(set(values))

values_map = dict(zip(list(values_set),range(len(values_set))))
for key,value in replace_map.items():
    values_map[key] =values_map[value]


pickle.dump(values_map,open(f'{model_dir}/txt_value_idx_map.pkl','wb'))

#### replace label with idx

In [32]:
for col in origin_discrete_cols+manul_discrete_cols:
    train_df[col] = train_df[col].apply(lambda x : values_map[x])

#### machine learning model: random forest

In [33]:
train_y =train_df['FRAUD_NONFRAUD']
label_map ={'Fraud':1,'Non-Fraud':0}
train_y =[label_map[i] for i in train_y]

train_x = train_df[origin_discrete_cols+manul_discrete_cols+continuous_features]

In [34]:
clf = RandomForestClassifier(max_depth=10, random_state =2021)
clf.fit(train_x, train_y)

# train dataset
predict_train_y = clf.predict(train_x)
predict_train_y =predict_train_y.tolist()
print(f"....random forest classification report---training dataset")
print('\n')
print(classification_report(y_true=train_y,y_pred=predict_train_y))


....random forest classification report---training dataset


              precision    recall  f1-score   support

           0       0.98      0.99      0.98      9836
           1       0.98      0.95      0.96      4164

    accuracy                           0.98     14000
   macro avg       0.98      0.97      0.97     14000
weighted avg       0.98      0.98      0.98     14000



In [35]:
importance = clf.feature_importances_.tolist()

In [36]:
bottom_cols =['ALERT_TRGR_CD','CUST_STATE_bin_10_feature','TRAN_TS_hour',
              'PWD_UPDT_TS_day','AUTHC_PRIM_TYPE_CD','AUTHC_PRIM_TYPE_CD',
             'AUTHC_SCNDRY_STAT_TXT','CUST_AGE']

In [37]:
train_y =train_df['FRAUD_NONFRAUD']
label_map ={'Fraud':1,'Non-Fraud':0}
train_y =[label_map[i] for i in train_y]

filterd_features =list(set(origin_discrete_cols+manul_discrete_cols+continuous_features) -set(bottom_cols))

train_x = train_df[filterd_features]

clf = RandomForestClassifier(max_depth=10, random_state =2021)
clf.fit(train_x, train_y)

# train dataset
predict_train_y = clf.predict(train_x)
predict_train_y =predict_train_y.tolist()
print(f"....random forest classification report---training dataset")
print('\n')
print(classification_report(y_true=train_y,y_pred=predict_train_y))


....random forest classification report---training dataset


              precision    recall  f1-score   support

           0       0.98      0.99      0.98      9836
           1       0.97      0.95      0.96      4164

    accuracy                           0.98     14000
   macro avg       0.98      0.97      0.97     14000
weighted avg       0.98      0.98      0.98     14000



### feature selected 

In [38]:
filterd_features

['DVC_TYPE_TXT',
 'WF_dvc_age',
 'PH_NUM_UPDT_TS_hour',
 'STATE_PRVNC_TXT_bin_10_feature',
 'PH_NUM_UPDT_TS_day',
 'PWD_UPDT_TS_hour',
 'ACCT_PRE_TRAN_AVAIL_BAL',
 'RGN_NAME_bin_10_feature',
 'OPEN_ACCT_CT',
 'TRAN_AMT',
 'CARR_NAME_bin_10_feature']